## Introduction


This notebook demonstrates the core functionality of pymatgen, including the core objects representing Elements, Species, Lattices, and Structures.


In [ ]:
# Uncomment the subsequent lines in this cell to install dependencies for Google Colab.
# !pip install pymatgen==2022.7.19

In [ ]:
import pymatgen.core as pmg

## Basic Element, Specie and Composition objects


Pymatgen contains a set of core classes to represent an Element, Specie and Composition. These objects contains useful properties such as atomic mass, ionic radii, etc. These core classes are loaded by default with pymatgen. An Element can be created as follows:


In [ ]:
si = pmg.Element("Si")
print(f"Atomic mass of Si is {si.atomic_mass}")
print(f"Si has a melting point of {si.melting_point}")
print(f"Ionic radii for Si: {si.ionic_radii}")

Atomic mass of Si is 28.0855 amu
Si has a melting point of 1687.0 K
Ionic radii for Si: {4: 0.54}


You can see that units are printed for atomic masses and ionic radii. Pymatgen comes with a complete system of managing units in pymatgen.core.unit. A Unit is a subclass of float that attaches units and handles conversions. For example,


In [ ]:
print("Atomic mass of Si in kg: {}".format(si.atomic_mass.to("kg")))

Atomic mass of Si in kg: 4.66370699549943e-26 kg


Please refer to the Units example for more information on units. Species are like Elements, except they have an explicit oxidation state. They can be used wherever Element is used for the most part.


In [ ]:
fe2 = pmg.Species("Fe", 2)
print(fe2.atomic_mass)
print(fe2.ionic_radius)

55.845 amu
0.92 ang


A Composition is essentially an **immutable** mapping of Elements/Species with amounts, and useful properties like molecular weight, get_atomic_fraction, etc. Note that you can conveniently either use an Element/Specie object or a string as keys (this is a feature).


In [ ]:
comp = pmg.Composition("Fe2O3")
print(f"Weight of Fe2O3 is {comp.weight}")
print("Amount of Fe in Fe2O3 is {}".format(comp["Fe"]))
print("Atomic fraction of Fe is {}".format(comp.get_atomic_fraction("Fe")))
print("Weight fraction of Fe is {}".format(comp.get_wt_fraction("Fe")))

Weight of Fe2O3 is 159.6882 amu
Amount of Fe in Fe2O3 is 2.0
Atomic fraction of Fe is 0.4
Weight fraction of Fe is 0.699425505453753 


## Lattice & Structure objects


A Lattice represents a Bravais lattice. Convenience static functions are provided for the creation of common lattice types from a minimum number of arguments.


In [ ]:
# Creates cubic Lattice with lattice parameter 4.2
lattice = pmg.Lattice.cubic(4.2)
print(lattice.parameters)

(4.2, 4.2, 4.2, 90.0, 90.0, 90.0)


A Structure object represents a crystal structure (lattice + basis). A Structure is essentially a list of PeriodicSites with the same Lattice. Let us now create a CsCl structure.


In [ ]:
structure = pmg.Structure(lattice, ["Cs", "Cl"], [[0, 0, 0], [0.5, 0.5, 0.5]])
print(f"Unit cell vol = {structure.volume}")
print(f"First site of the structure is {structure[0]}")

Unit cell vol = 74.08800000000001
First site of the structure is [0. 0. 0.] Cs


The Structure object contains many useful manipulation functions. Since Structure is essentially a list, it contains a simple pythonic API for manipulation its sites. Some examples are given below. Please note that there is an immutable version of Structure known as IStructure, for the use case where you really need to enforce that the structure does not change. Conversion between these forms of Structure can be performed using from_sites().


In [ ]:
structure.make_supercell([2, 2, 1])  # Make a 3 x 2 x 1 supercell of the structure
del structure[0]  # Remove the first site
structure.append("Na", [0, 0, 0])  # Append a Na atom.
structure[-1] = "Li"  # Change the last added atom to Li.
# Shift the first atom by 0.01 in fractional coordinates in the x-direction.
structure[0] = ("Cs", [0.01, 0.5, 0])
immutable_structure = pmg.IStructure.from_sites(structure)  # Create an immutable structure (cannot be modified).
print(immutable_structure)

Full Formula (Cs3 Li1 Cl4)
Reduced Formula: Cs3LiCl4
abc   :   8.400000   8.400000   4.200000
angles:  90.000000  90.000000  90.000000
Sites (8)
  #  SP       a     b    c
---  ----  ----  ----  ---
  0  Cs    0.01  0.5   0
  1  Cs    0.5   0     0
  2  Cs    0.5   0.5   0
  3  Cl    0.25  0.25  0.5
  4  Cl    0.25  0.75  0.5
  5  Cl    0.75  0.25  0.5
  6  Cl    0.75  0.75  0.5
  7  Li    0     0     0


## Basic analyses


Pymatgen provides many analyses functions for Structures. Some common ones are given below.


In [ ]:
# Determining the symmetry
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

finder = SpacegroupAnalyzer(structure)
print(f"The spacegroup is {finder.get_space_group_symbol()}")

The spacegroup is Pmm2


We also have an extremely powerful structure matching tool.


In [ ]:
from pymatgen.analysis.structure_matcher import StructureMatcher

# Let's create two structures which are the same topologically, but with different elements, and one lattice is larger.
s1 = pmg.Structure(lattice, ["Cs", "Cl"], [[0, 0, 0], [0.5, 0.5, 0.5]])
s2 = pmg.Structure(pmg.Lattice.cubic(5), ["Rb", "F"], [[0, 0, 0], [0.5, 0.5, 0.5]])
m = StructureMatcher()
print(
    m.fit_anonymous(s1, s2)
)  # Returns a mapping which maps s1 and s2 onto each other. Strict element fitting is also available.

True


## Input/output


Pymatgen also provides IO support for various file formats in the pymatgen.io package. A convenient set of read_structure and write_structure functions are also provided which auto-detects several well-known formats.


In [ ]:
# Convenient IO to various formats. Format is intelligently determined from file name and extension.
structure.to(filename="POSCAR")
structure.to(filename="CsCl.cif")

# Or if you just supply fmt, you simply get a string.
print(structure.to(fmt="poscar"))
print(structure.to(fmt="cif"))

Cs3 Li1 Cl4
1.0
8.400000 0.000000 0.000000
0.000000 8.400000 0.000000
0.000000 0.000000 4.200000
Cs Cl Li
3 4 1
direct
0.010000 0.500000 0.000000 Cs
0.500000 0.000000 0.000000 Cs
0.500000 0.500000 0.000000 Cs
0.250000 0.250000 0.500000 Cl
0.250000 0.750000 0.500000 Cl
0.750000 0.250000 0.500000 Cl
0.750000 0.750000 0.500000 Cl
0.000000 0.000000 0.000000 Li

# generated using pymatgen
data_Cs3LiCl4
_symmetry_space_group_name_H-M   'P 1'
_cell_length_a   8.40000000
_cell_length_b   8.40000000
_cell_length_c   4.20000000
_cell_angle_alpha   90.00000000
_cell_angle_beta   90.00000000
_cell_angle_gamma   90.00000000
_symmetry_Int_Tables_number   1
_chemical_formula_structural   Cs3LiCl4
_chemical_formula_sum   'Cs3 Li1 Cl4'
_cell_volume   296.35200000
_cell_formula_units_Z   1
loop_
 _symmetry_equiv_pos_site_id
 _symmetry_equiv_pos_as_xyz
  1  'x, y, z'
loop_
 _atom_site_type_symbol
 _atom_site_label
 _atom_site_symmetry_multiplicity
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract

In [ ]:
# Reading a structure from a file.
structure = pmg.Structure.from_file("POSCAR")

The vaspio_set module provides a means o obtain a complete set of VASP input files for performing calculations. Several useful presets based on the parameters used in the Materials Project are provided.


In [ ]:
from pymatgen.io.vasp.sets import MPRelaxSet

vasp_inputs = MPRelaxSet(structure)
# Write complete set of input files for structure to the disk
vasp_inputs.write_input("./vasp_relaxation")

/Users/shyue/repos/pymatgen/pymatgen/io/vasp/inputs.py:1823: UnknownPotcarWarning: POTCAR with symbol Cs_sv has metadata that does not match                          any VASP POTCAR known to pymatgen. The data in this                          POTCAR is known to match the following functionals:                          ['PBE']
  warnings.warn(
/Users/shyue/repos/pymatgen/pymatgen/io/vasp/inputs.py:1816: UnknownPotcarWarning: POTCAR data with symbol Li_sv does not match any VASP POTCAR known to pymatgen. There is a possibility your POTCAR is corrupted or that the pymatgen database is incomplete.
  warnings.warn(
/Users/shyue/repos/pymatgen/pymatgen/io/vasp/inputs.py:1823: UnknownPotcarWarning: POTCAR with symbol Cl has metadata that does not match                          any VASP POTCAR known to pymatgen. The data in this                          POTCAR is known to match the following functionals:                          ['PBE']
  warnings.warn(
/Users/shyue/repos/pymatgen/pymatgen/io/

### This concludes this pymatgen tutorial. Please explore the usage pages on pymatgen.org for more information.
